In [1]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, ElasticNetCV, ElasticNet, RidgeCV, LassoCV, HuberRegressor
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.compose import TransformedTargetRegressor


%matplotlib inline
plt.style.use('ggplot')

In [2]:
str_to_ndarray = lambda x: np.fromstring(x, sep=' ')

path = os.path.join('..', '..', 'data', 'KG_combin.csv')
kg_data = pd.read_csv(path, converters={'eigvals': str_to_ndarray})

for q in range(14):#nondeg_minlen):
    kg_data['omega2_' + str(q)] = kg_data['eigvals'].apply(lambda arr: arr[6 + q]) / kg_data['rho']

kg_data = kg_data.drop(columns=['eigvals'])

In [3]:
kg_data

,K,G,rho,dx,dy,dz,shape,omega2_0,omega2_1,omega2_2,...,omega2_4,omega2_5,omega2_6,omega2_7,omega2_8,omega2_9,omega2_10,omega2_11,omega2_12,omega2_13
0,0.3,0.300000,0.2,0.1,0.1,0.1,cone,363.812512,363.812512,616.861943,...,801.530462,1202.741616,1202.741616,1360.593158,1360.593158,1391.954446,1391.954446,1598.471054,1598.471054,2043.801060
1,0.3,1.057143,0.2,0.1,0.1,0.1,cone,1080.357105,1237.165522,1237.165522,...,2651.506483,2768.883511,2768.883511,3218.011759,3218.011759,3335.602641,4738.235859,4738.235859,4917.977790,4917.977790
2,0.3,1.814286,0.2,0.1,0.1,0.1,cone,1279.463309,2075.013838,2075.013838,...,3628.548353,3628.548353,3915.845931,3915.845931,4683.416721,4683.416721,6711.904416,6711.904416,8050.533407,8050.533407
3,0.3,2.571429,0.2,0.1,0.1,0.1,cone,1402.145793,2886.110200,2886.110200,...,4161.645389,4161.645389,4597.790620,4597.790620,6539.611577,6539.611577,7540.838290,7540.838290,10185.591908,11261.941102
4,0.3,3.328571,0.2,0.1,0.1,0.1,cone,1488.362993,3668.498967,3668.498967,...,4360.684462,4360.684462,5436.294393,5436.294393,8067.005188,8067.005188,8300.251487,8300.251487,11831.588014,12960.953559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229371,5.6,2.571429,10.0,1.0,1.0,1.0,tetrahedron,1.776489,1.992767,1.992767,...,2.899190,4.501326,4.501326,4.935697,6.125115,7.256192,8.070548,8.070548,9.253246,9.253246
229372,5.6,3.328571,10.0,1.0,1.0,1.0,tetrahedron,2.213418,2.533433,2.533433,...,3.643310,5.720904,5.720904,6.363125,7.760717,9.045892,10.325101,10.325101,11.806433,11.806433
229373,5.6,4.085714,10.0,1.0,1.0,1.0,tetrahedron,2.618126,3.053709,3.053709,...,4.345456,6.901332,6.901332,7.772311,9.335054,10.704454,12.520110,12.520110,14.275421,14.275421
229374,5.6,4.842857,10.0,1.0,1.0,1.0,tetrahedron,2.993823,3.554073,3.554073,...,5.009227,8.046839,8.046839,9.157921,10.858081,12.246230,14.653307,14.653307,16.657100,16.657100


In [ ]:
a

In [4]:
kg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229376 entries, 0 to 229375
Data columns (total 21 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   K          229376 non-null  float64
 1   G          229376 non-null  float64
 2   rho        229376 non-null  float64
 3   dx         229376 non-null  float64
 4   dy         229376 non-null  float64
 5   dz         229376 non-null  float64
 6   shape      229376 non-null  object 
 7   omega2_0   229376 non-null  float64
 8   omega2_1   229376 non-null  float64
 9   omega2_2   229376 non-null  float64
 10  omega2_3   229376 non-null  float64
 11  omega2_4   229376 non-null  float64
 12  omega2_5   229376 non-null  float64
 13  omega2_6   229376 non-null  float64
 14  omega2_7   229376 non-null  float64
 15  omega2_8   229376 non-null  float64
 16  omega2_9   229376 non-null  float64
 17  omega2_10  229376 non-null  float64
 18  omega2_11  229376 non-null  float64
 19  omega2_12  229376 non-n

# PIPELINE

In [11]:
 # Separación de features y target
X = kg_data.drop(['K', 'G'], axis=1)
y = kg_data['K']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1)

sqrt_columns = ['rho', 'dx', 'dy', 'dz']
omega_columns = [f'omega2_{i}' for i in range(14)]
categorical_columns = ['shape']

# Definición de transformaciones de features
feature_transformer = ColumnTransformer(transformers=[
    ('sqrt', FunctionTransformer(np.sqrt), sqrt_columns),
    ('log', FunctionTransformer(np.log), omega_columns),
    ('onehot', OneHotEncoder(drop='first'), categorical_columns)
], remainder='drop')

# Creación del pipeline
pipeline = Pipeline(steps=[
    ('feature_transformation', feature_transformer),
    ('scaling', MinMaxScaler()),
    ('regression', LinearRegression(fit_intercept=True))
])

# TransformedTargetRegressor para transformar la variable objetivo (np.sqrt(K))
model = TransformedTargetRegressor(regressor=pipeline, func=np.sqrt, inverse_func=np.square)

model.fit(X_train, y_train)

# Predice los valores automáticamente con la transformación inversa
y_pred = model.predict(X_test)

# Métricas sobre los datos originales
r2 = r2_score(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'R2: {r2:.3f}')
print(f'RMSE: {rmse:.3f}')
print(f'MAE: {mae:.3f}')
print(f'MAPE: {mape:.3f}')

R2: 0.021
RMSE: 1.716
MAE: 1.467
MAPE: 1.296


This Stack Overflow post explains the discrepancy between scikit-learn and statsmodel $R^2$ https://stackoverflow.com/questions/54614157/scikit-learn-statsmodels-which-r-squared-is-correct. It also mentions the idea of using different metrics for prediction models instead of $R^2$.

# MANUAL (comprobación)

In [12]:
# Separación de features y target
X = kg_data.drop(['K', 'G'], axis=1)
y = kg_data['K']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, random_state=1)

sqrt_columns = ['rho', 'dx', 'dy', 'dz']
omega_columns = [f'omega2_{i}' for i in range(14)]
categorical_columns = ['shape']

In [13]:
# aplicar transformacion a columnas
X_train[sqrt_columns] = np.sqrt(X_train[sqrt_columns])
X_train[omega_columns] = np.log(X_train[omega_columns])
# one hot encoding
X_train = pd.get_dummies(X_train, columns=categorical_columns, drop_first=True)
# transformacion a target
y_train = np.sqrt(y_train)

# lo mismo a test
X_test[sqrt_columns] = np.sqrt(X_test[sqrt_columns])
X_test[omega_columns] = np.log(X_test[omega_columns])
# one hot encoding
X_test = pd.get_dummies(X_test, columns=categorical_columns, drop_first=True)
y_test = np.sqrt(y_test)


In [14]:
# hacer regresion
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)

# predecir
y_pred = model.predict(X_test)

# invertir la transformacion
y_pred = np.square(y_pred)
y_test = np.square(y_test)

# metricas
r2 = r2_score(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'R2: {r2:.3f}')
print(f'RMSE: {rmse:.3f}')
print(f'MAE: {mae:.3f}')
print(f'MAPE: {mape:.3f}')

R2: 0.021
RMSE: 1.716
MAE: 1.467
MAPE: 1.296


# Pipeline + CV

In [25]:
# Separación de features y target
X = kg_data.drop(['K', 'G'], axis=1)
y = kg_data['K']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) #, random_state=1)

sqrt_columns = ['rho', 'dx', 'dy', 'dz']
omega_columns = [f'omega2_{i}' for i in range(14)]
categorical_columns = ['shape']

# Definición de transformaciones de features
feature_transformer = ColumnTransformer(transformers=[
    ('sqrt', FunctionTransformer(np.sqrt), sqrt_columns),
    ('log', FunctionTransformer(np.log), omega_columns),
    ('onehot', OneHotEncoder(drop='first'), categorical_columns)
], remainder='drop')

# Creación del pipeline con ElasticNetCV
elasticnet_cv = ElasticNetCV(fit_intercept=True, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=5, max_iter=10000)

# Probar con GridSearchCV
"""
param_grid = {
    'l1_ratio' : [.1, .5, .7, .9, .95, .99, 1],
    'alpha' : np.logspace(-4, 4, 20)
}
elasticnet_cv = GridSearchCV(estimator=ElasticNet(fit_intercept=True, max_iter=100000), param_grid=param_grid, 
                           scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)

# Mejores parámetros encontrados
best_alpha = model.regressor_.named_steps['regression'].best_estimator_.alpha
best_l1_ratio = model.regressor_.named_steps['regression'].best_estimator_.l1_ratio
print(f'Best alpha: {best_alpha:.3f}')
print(f'Best l1_ratio: {best_l1_ratio:.3f}')
# más demorado pero mismos resultados:
# Best alpha: 0.000
# Best l1_ratio: 1.000
"""

# probar RidgeCV
#ridge_cv = RidgeCV(fit_intercept=True, alphas=(0, 10.0, 20.0), cv=5, scoring='neg_root_mean_squared_error') 

# probar LassoCV
# lasso_cv = LassoCV(fit_intercept=True, cv=5, max_iter=100000)

# Creación del pipeline
pipeline = Pipeline(steps=[
    ('feature_transformation', feature_transformer),
    ('scaling', MinMaxScaler()),
    ('regression', elasticnet_cv)
])


# TransformedTargetRegressor para transformar la variable objetivo (np.sqrt(K))
model = TransformedTargetRegressor(regressor=pipeline, func=np.sqrt, inverse_func=np.square)

model.fit(X_train, y_train)

# Predice los valores automáticamente con la transformación inversa
y_pred = model.predict(X_test)

# Métricas sobre los datos originales
r2 = r2_score(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'R2: {r2:.3f}')
print(f'RMSE: {rmse:.3f}')
print(f'MAE: {mae:.3f}')
print(f'MAPE: {mape:.3f}')

# Mejores parámetros encontrados
best_alpha = model.regressor_.named_steps['regression'].alpha_
best_l1_ratio = model.regressor_.named_steps['regression'].l1_ratio_
print(f'Best alpha: {best_alpha}')
print(f'Best l1_ratio: {best_l1_ratio}')

R2: 0.019
RMSE: 1.718
MAE: 1.468
MAPE: 1.293
Best alpha: 8.981032262332428e-06
Best l1_ratio: 1.0


l1_ratio <= 0.01 is not reliable

``alpha = 0`` is equivalent to an ordinary least square

l1_ratio = 1 is the lasso penalty

In [28]:
# Separación de features y target
X = kg_data.drop(['K', 'G'], axis=1)
y = kg_data['K']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) #, random_state=1)

sqrt_columns = ['rho', 'dx', 'dy', 'dz']
omega_columns = [f'omega2_{i}' for i in range(14)]
categorical_columns = ['shape']

# Definición de transformaciones de features
feature_transformer = ColumnTransformer(transformers=[
    ('sqrt', FunctionTransformer(np.sqrt), sqrt_columns),
    ('log', FunctionTransformer(np.log), omega_columns),
    ('onehot', OneHotEncoder(drop='first'), categorical_columns)
], remainder='drop')

# Creación del pipeline
pipeline = Pipeline(steps=[
    ('feature_transformation', feature_transformer),
    ('scaling', MinMaxScaler()),
    ('regression', ElasticNet(fit_intercept=True, max_iter=10000)
)
])

# TransformedTargetRegressor para transformar la variable objetivo (np.sqrt(K))
target_regressor = TransformedTargetRegressor(regressor=pipeline, func=np.sqrt, inverse_func=np.square)

param_grid = {
    'regressor__regression__l1_ratio' : [.1, .5, .7, .9, .95, .99, 1],
    'regressor__regression__alpha' : np.logspace(-4, 4, 20)
}
model = GridSearchCV(estimator=target_regressor, param_grid=param_grid, 
                           scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)

model.fit(X_train, y_train)

# Predice los valores automáticamente con la transformación inversa
y_pred = model.predict(X_test)

# Métricas sobre los datos originales
r2 = r2_score(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'R2: {r2:.3f}')
print(f'RMSE: {rmse:.3f}')
print(f'MAE: {mae:.3f}')
print(f'MAPE: {mape:.3f}')

# Mejores parámetros encontrados
best_l1_ratio = model.best_params_['regressor__regression__l1_ratio']
best_alpha = model.best_params_['regressor__regression__alpha']
print(f'Best l1_ratio: {best_l1_ratio}')
print(f'Best alpha: {best_alpha}')

R2: 0.009
RMSE: 1.727
MAE: 1.479
MAPE: 1.312
Best l1_ratio: 1
Best alpha: 0.0001


# Huber

In [4]:
# Separación de features y target
X = kg_data.drop(['K', 'G'], axis=1)
y = kg_data['K']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) #, random_state=1)

sqrt_columns = ['rho', 'dx', 'dy', 'dz']
omega_columns = [f'omega2_{i}' for i in range(14)]
categorical_columns = ['shape']

# Definición de transformaciones de features
feature_transformer = ColumnTransformer(transformers=[
    ('sqrt', FunctionTransformer(np.sqrt), sqrt_columns),
    ('log', FunctionTransformer(np.log), omega_columns),
    ('onehot', OneHotEncoder(drop='first'), categorical_columns)
], remainder='drop')


# Creación del pipeline
pipeline = Pipeline(steps=[
    ('feature_transformation', feature_transformer),
    ('scaling', MinMaxScaler()),
    ('regression', HuberRegressor(fit_intercept=True, max_iter=10000))
])


# TransformedTargetRegressor para transformar la variable objetivo (np.sqrt(K))
target_regressor = TransformedTargetRegressor(regressor=pipeline, func=np.sqrt, inverse_func=np.square)

param_grid = {
    'regressor__regression__epsilon': [1, 1.35, 1.5, 1.8],
    'regressor__regression__alpha': [0, 0.0001, 0.001, 0.01]
}

model = GridSearchCV(
    estimator= target_regressor,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    scoring='neg_root_mean_squared_error',   
)

model.fit(X_train, y_train)

# Predice los valores automáticamente con la transformación inversa
y_pred = model.predict(X_test)

# Métricas sobre los datos originales
r2 = r2_score(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'R2: {r2:.3f}')
print(f'RMSE: {rmse:.3f}')
print(f'MAE: {mae:.3f}')
print(f'MAPE: {mape:.3f}')

# Mejores parámetros encontrados
best_alpha = model.best_params_['regressor__regression__alpha']
best_epsilon = model.best_params_['regressor__regression__epsilon']
print(f'Best alpha: {best_alpha}')
print(f'Best epsilon: {best_epsilon}')

R2: 0.041
RMSE: 1.698
MAE: 1.453
MAPE: 1.382
Best alpha: 0
Best epsilon: 1
